In [166]:
from bs4 import BeautifulSoup
import requests
import json
import os
import re
import gc

In [20]:
ms_curr_url = 'https://pubsonline.informs.org/toc/mnsc/65/2'

In [21]:
s = requests.Session()
# s.post(some_url, data={'username':'','password':''})
r = s.get(ms_curr_url)
soup = BeautifulSoup(r.text, 'html')

## issue header

In [165]:
soup.find('div','cover-image__details').h2.text

'Volume 65, Issue 2'

In [163]:
soup.find('div').find('div','volume--date').text

'February 2019'

In [164]:
soup.find('div','volume--pages').text

'Pages v-vi, 459-954'

## issue items

In [153]:
issue_item = soup.find_all('div', 'issue-item')

In [170]:
len(issue_item)

27

### get title 

In [154]:
issue_item[1].h5.a.text

'Do General Managerial Skills Spur Innovation?'

### get authors 

In [89]:
for _ in issue_item[1].find_all('a', 'entryAuthor'):
    print(_.text) 

Cláudia Custódio 
Miguel A. Ferreira 
Pedro Matos 


### get date 

In [146]:
issue_item[1].find('div', 'toc-item__detail').find_all('p')[1].text[17:]

'September 25, 2017'

### get abstract

In [149]:
issue_item[1].find('span', 'hlFld-Abstract').text

'We show that firms with chief executive officers (CEOs) who gain general managerial skills over their lifetime of work experience produce more patents. We address the potential endogenous CEO–firm matching bias using firm–CEO fixed effects and variation ...'

### get link

In [233]:
issue_item[1].find('p',recursive=False).a['href']

'https://doi.org/10.1287/mnsc.2017.2828'

## to json

In [171]:
# to dict
ms_dict = {'issue':soup.find('div','cover-image__details').h2.text, 
           'date':soup.find('div').find('div','volume--date').text, 
           'page':soup.find('div','volume--pages').text, 
           'issue_item_num': len(soup.find_all('div', 'issue-item'))}

## to dot md

In [280]:
md_file = open('201902MS1.md','w')

# volume and issue
md_file.write('# {}\n'.format(soup.find('div','cover-image__details').h2.text))
# date
md_file.write('\n- {}'.format(soup.find('div').find('div','volume--date').text))
# pages
md_file.write('\n- {}'.format(soup.find('div','volume--pages').text))

issue_item = soup.find_all('div', 'issue-item')
# len
md_file.write('\n- {} Articles\n'.format(len(issue_item)))
md_file.write('\n## {}\n\n'.format('Issue Items:'))

for c, item in enumerate(issue_item):
    md_file.write('### {}. {}\n'.format(c+1, item.h5.a.text))
    md_file.write('#### Author(s):\n'.format())
    for author in item.find_all('a', 'entryAuthor'):
        md_file.write('- {}\n'.format(author.text))
    md_file.write('#### Date:\n- {}\n'.format(item.find('div', 'toc-item__detail').find_all('p')[1].text[17:]))
    md_file.write('#### Abstract:\n{}\n'.format(item.find('span', 'hlFld-Abstract').text))
    md_file.write('#### Link:\n- {}\n'.format(item.find('p',recursive=False).a['href']))
    
    md_file.write('\n')

md_file.close()

## function

In [334]:
def get_ms(volume, issue):
    base_url = 'https://pubsonline.informs.org/toc/mnsc/'
    s = requests.Session()
    r = s.get(base_url+str(volume)+'/'+str(issue))
    soup = BeautifulSoup(r.text, 'html')
    md_file = open('Volume {:02d} - Issue {:02d}.md'.format(volume,issue),'w')

    # volume and issue
    md_file.write('# {}\n'.format(soup.find('div','cover-image__details').h2.text))
    # date
    md_file.write('\n- {}'.format(soup.find('div').find('div','volume--date').text))
    # pages
    md_file.write('\n- {}'.format(soup.find('div','volume--pages').text))

    issue_item = soup.find_all('div', 'issue-item')
    # len
    md_file.write('\n- {} Articles\n'.format(len(issue_item)))
    md_file.write('\n## {}\n\n'.format('Issue Items:'))

    for c, item in enumerate(issue_item):
        md_file.write('### {}. {}\n'.format(c+1, item.h5.a.text))
        md_file.write('#### Author(s):\n'.format())
        for author in item.find_all('a', 'entryAuthor'):
            md_file.write('- {}\n'.format(author.text))
        md_file.write('#### Date:\n- {}\n'.format(item.find('div', 'toc-item__detail').find_all('p')[1].text[17:]))
        try:
            md_file.write('#### Abstract:\n{}\n'.format(item.find('span', 'hlFld-Abstract').text))
        except:
            md_file.write('#### Abstract:\n{}\n'.format(''))
        md_file.write('#### Link:\n- {}\n'.format(item.find('p',recursive=False).a['href']))

        md_file.write('\n')

    md_file.close()

In [242]:
base_url = 'https://pubsonline.informs.org/toc/mnsc/'

In [304]:
file_dir_name = 'MS'

if os.getcwd()[13:] == 'light-speed-engine':
    if os.path.isdir(file_dir_name):
        os.chdir(file_dir_name)
    else:
        os.mkdir(file_dir_name)
        os.chdir(file_dir_name)
else:
    print('ERORR: Not in the right folder')

In [346]:
os.getcwd()

'/Users/dyang/light-speed-engine/MS'

In [345]:
os.chdir('..')

In [302]:
os.chdir('light-speed-engine')

In [347]:
for i in range(56,65):
    folder = 'Volume {:02d}'.format(i)
    if os.path.isdir(folder):
        os.chdir(folder)
    else:
        os.mkdir(folder)
        os.chdir(folder)
    for j in range(1,13):
        print('getting volune {:02d} issue {:02d}'.format(i,j))
        get_ms(i,j)
    os.chdir('..')

getting volune 56 issue 01
getting volune 56 issue 02
getting volune 56 issue 03
getting volune 56 issue 04
getting volune 56 issue 05
getting volune 56 issue 06
getting volune 56 issue 07
getting volune 56 issue 08
getting volune 56 issue 09
getting volune 56 issue 10
getting volune 56 issue 11
getting volune 56 issue 12
getting volune 57 issue 01
getting volune 57 issue 02
getting volune 57 issue 03
getting volune 57 issue 04
getting volune 57 issue 05
getting volune 57 issue 06
getting volune 57 issue 07
getting volune 57 issue 08
getting volune 57 issue 09
getting volune 57 issue 10
getting volune 57 issue 11
getting volune 57 issue 12
getting volune 58 issue 01
getting volune 58 issue 02
getting volune 58 issue 03
getting volune 58 issue 04
getting volune 58 issue 05
getting volune 58 issue 06
getting volune 58 issue 07
getting volune 58 issue 08
getting volune 58 issue 09
getting volune 58 issue 10
getting volune 58 issue 11
getting volune 58 issue 12
getting volune 59 issue 01
g